# Great Expectations（GE） によるノートブック型環境（Databricks）でのデータ品質保証を実行する方法のまとめ

## Great Expectations とは

Great Expectations (GE) とは、データに対する検証、ドキュメント化、および、プロファイリングにより、データ品質の保証と改善を支援する OSS の Python ライブラリである。


GE に関する基本的な記事として、次の記事が公開されている。

| #    | 記事                                                       | 概要                   |
| ---- | ------------------------------------------------------------ | ---------------------- |
| 1    | [Welcome](https://docs.greatexpectations.io/docs/)           | 概要                   |
| 2    | [Getting started with Great Expectations](https://docs.greatexpectations.io/docs/tutorials/getting_started/tutorial_overview) | チュートリアル         |
| 3    | [Glossary of Terms](https://docs.greatexpectations.io/docs/glossary) | 用語集                 |
| 4    | [Customize your deployment Great Expectations](https://docs.greatexpectations.io/docs/reference/customize_your_deployment) | 利用するための考慮事項 |
| 5    | [Explore Expectations](https://greatexpectations.io/expectations/) | Expectation 一覧       |
| 6    | [Community Page • Great Expectations](https://greatexpectations.io/community) | コミュニティ関連       |
| 7    | [Case studies from Great Expectations](https://greatexpectations.io/case-studies/) | ケーススタディ         |



GE を利用する基本的な手順は次のようになっている。データに対する品質保証条件を Expectations として定義に基づき、データソースへの検証を行い、検証結果をドキュメント化することができる。

1. セットアップ
2. データへの接続
3. Expectations の作成
4. データの検証


検証できるデータソースには次のものがある。

- SQLAlchemy 経由によるデータベース
- Pandas Dataframe
- Spark Dataframe

データソースごとに利用できる Expection が異なり、次のドキュメントにて整理されている。

- [Expectation implementations by backend | Great Expectations](https://docs.greatexpectations.io/docs/reference/expectations/implemented_expectations/)

GE を利用する方法として、CLIによる方法とノートブック型環境による方法がある。ノートブック型環境による方法を実施する際には、次のドキュメントが参考となる。

-   [How to instantiate a Data Context without a yml file](https://docs.greatexpectations.io/docs/guides/setup/configuring_data_contexts/how_to_instantiate_a_data_context_without_a_yml_file/)
-   [How to quickly explore Expectations in a notebook | Great Expectations](https://docs.greatexpectations.io/docs/guides/miscellaneous/how_to_quickly_explore_expectations_in_a_notebook/)
-   [How to pass an in-memory DataFrame to a Checkpoint | Great Expectations](https://docs.greatexpectations.io/docs/guides/validation/checkpoints/how_to_pass_an_in_memory_dataframe_to_a_checkpoint/)


データ検証後に、ドキュメントを作成するだけでなく、次のような [Action](https://docs.greatexpectations.io/docs/terms/action) を設定可能。

-   [How to trigger Email as a Validation Action](https://docs.greatexpectations.io/docs/guides/validation/validation_actions/how_to_trigger_email_as_a_validation_action)
-   [How to collect OpenLineage metadata using a Validation Action](https://docs.greatexpectations.io/docs/guides/validation/validation_actions/how_to_collect_openlineage_metadata_using_a_validation_action)
-   [How to trigger Opsgenie notifications as a Validation Action](https://docs.greatexpectations.io/docs/guides/validation/validation_actions/how_to_trigger_opsgenie_notifications_as_a_validation_action)
-   [How to trigger Slack notifications as a Validation Action](https://docs.greatexpectations.io/docs/guides/validation/validation_actions/how_to_trigger_slack_notifications_as_a_validation_action)
-   [How to update Data Docs after validating a Checkpoint](https://docs.greatexpectations.io/docs/guides/validation/validation_actions/how_to_update_data_docs_as_a_validation_action)

## Greate Exceptions を利用するための事前準備

### 1. Great Expectations のインストール

In [0]:
%pip install great_expectations -q

Python interpreter will be restarted.
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d40ba107-8afd-4b1f-a0d3-815f086c1d59/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

### 2. データ（データフレーム）の準備

In [0]:
schema = '''
`VendorID` INT,
`tpep_pickup_datetime` TIMESTAMP,
`tpep_dropoff_datetime` TIMESTAMP,
`passenger_count` INT,
`trip_distance` DOUBLE,
`RatecodeID` INT,
`store_and_fwd_flag` STRING,
`PULocationID` INT,
`DOLocationID` INT,
`payment_type` INT,
`fare_amount` DOUBLE,
`extra` DOUBLE,
`mta_tax` DOUBLE,
`tip_amount` DOUBLE,
`tolls_amount` DOUBLE,
`improvement_surcharge` DOUBLE,
`total_amount` DOUBLE,
`congestion_surcharge` DOUBLE
'''

src_files = [
    "/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-01.csv.gz",
#     "/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-02.csv.gz",
]

tgt_df = (
    spark
    .read
    .format("csv")
    .schema(schema)
    .option("header", "true")
    .option("inferSchema", "false")
    .load(src_files)
)

## 基本的なデータ品質検証の実施

次の記事を参考にしている。

- [How to Use Great Expectations in Databricks | Great Expectations](https://docs.greatexpectations.io/docs/deployment_patterns/how_to_use_great_expectations_in_databricks/)

### Great Expectations のセットアップ

In [0]:
import datetime

from ruamel import yaml

import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)

In [0]:
# 検証結果を永続的に保存する場合には、`/dbfs`ディレクトリに配置すること
root_directory = "/tmp/great_expectations"
root_directory_in_spark_api = f"file:{root_directory}"

In [0]:
# root_directory の初期化
dbutils.fs.rm(root_directory_in_spark_api, True)

try:
    # ディレクトリを確認
    display(dbutils.fs.ls(root_directory_in_spark_api))
except:
    print('Directory is empty.')

Directory is empty.

In [0]:
# Great expectaions 利用時のエントリーポイントである Data Context を定義
# https://docs.greatexpectations.io/docs/terms/data_context/

# great_expectations.yml を参照せずに定義を実施
data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = BaseDataContext(project_config=data_context_config)

# 利用状況の情報共有を提供を停止
# https://docs.greatexpectations.io/docs/reference/anonymous_usage_statistics/
context.anonymous_usage_statistics.enabled = False

In [0]:
# ディレクトリを確認
display(dbutils.fs.ls(root_directory_in_spark_api))

path,name,size
file:/tmp/great_expectations/uncommitted/,uncommitted/,4096
file:/tmp/great_expectations/profilers/,profilers/,4096
file:/tmp/great_expectations/expectations/,expectations/,4096
file:/tmp/great_expectations/checkpoints/,checkpoints/,4096


### データへの接続

In [0]:
datasource_name = "taxi_datasource"
dataconnector_name = "databricks_df"
data_asset_name = "nyctaxi_tripdata_yellow_yellow_tripdata"
tgt_deploy_env = "prod"

In [0]:
datasource_config = {
    # データソースを定義
    # https://docs.greatexpectations.io/docs/terms/datasource
    "name": datasource_name,
    "class_name": "Datasource",

    # execution_engine を定義
    # https://docs.greatexpectations.io/docs/terms/execution_engine/
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "SparkDFExecutionEngine",
    },

    # データコネクターを定義
    # https://docs.greatexpectations.io/docs/terms/data_connector/
    "data_connectors": {
        dataconnector_name: {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "some_key_maybe_pipeline_stage",
                "some_other_key_maybe_run_id",
            ],
        }
    },
}
context.add_datasource(**datasource_config)

Out[8]: <great_expectations.datasource.new_datasource.Datasource at 0x7f3892631e50>

In [0]:
batch_request = RuntimeBatchRequest(
    datasource_name=datasource_name,
    data_connector_name=dataconnector_name,
    data_asset_name = data_asset_name,
    batch_identifiers={
        "some_key_maybe_pipeline_stage": tgt_deploy_env,
        "some_other_key_maybe_run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": tgt_df},
)

### Expectations を作成

In [0]:
expectation_suite_name = "nyctaxi_tripdata_yellow_yellow_tripdata"

In [0]:
context.create_expectation_suite(
    expectation_suite_name=expectation_suite_name,
    overwrite_existing=True,
)

Out[11]: {
 "expectations": [],
 "data_asset_type": null,
 "meta": {
 "great_expectations_version": "0.15.18"
 },
 "expectation_suite_name": "nyctaxi_tripdata_yellow_yellow_tripdata",
 "ge_cloud_id": null
}

In [0]:
# expectations にファイルが作成されたことを確認
expectations_file_path = f'{root_directory_in_spark_api}/expectations/{expectation_suite_name}.json'
print(dbutils.fs.head(expectations_file_path))

{
 "data_asset_type": null,
 "expectation_suite_name": "nyctaxi_tripdata_yellow_yellow_tripdata",
 "expectations": [],
 "ge_cloud_id": null,
 "meta": {
 "great_expectations_version": "0.15.18"
 }
}

In [0]:
# Validator
# https://docs.greatexpectations.io/docs/terms/validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

# exception を追加する際に検証を実施しないように設定
validator.interactive_evaluation = False

In [0]:
# exception を追加
_ = validator.expect_column_values_to_not_be_null(
    column="passenger_count",
)

_ = validator.expect_column_values_to_be_between(
    column="congestion_surcharge",
    min_value=0,
    max_value=1000,
    meta={
        "notes": {
            "format": "markdown",
            "content": "Example notes about this expectation. **Markdown** `Supported`."
        }
    },
)

_ = validator.expect_column_values_to_be_between(
    column="passenger_count",
    min_value=0,
    max_value=1000,
)

In [0]:
# expectations を保存
validator.save_expectation_suite(discard_failed_expectations=False)

In [0]:
# expectations がファイルに追記されたことを確認
print(dbutils.fs.head(expectations_file_path))

{
 "data_asset_type": null,
 "expectation_suite_name": "nyctaxi_tripdata_yellow_yellow_tripdata",
 "expectations": [
 {
 "expectation_type": "expect_column_values_to_not_be_null",
 "kwargs": {
 "column": "passenger_count"
 },
 "meta": {}
 },
 {
 "expectation_type": "expect_column_values_to_be_between",
 "kwargs": {
 "column": "congestion_surcharge",
 "max_value": 1000,
 "min_value": 0
 },
 "meta": {
 "notes": {
 "content": "Example notes about this expectation. **Markdown** `Supported`.",
 "format": "markdown"
 }
 }
 },
 {
 "expectation_type": "expect_column_values_to_be_between",
 "kwargs": {
 "column": "passenger_count",
 "max_value": 1000,
 "min_value": 0
 },
 "meta": {}
 }
 ],
 "ge_cloud_id": null,
 "meta": {
 "great_expectations_version": "0.15.18"
 }
}

### データの検証

In [0]:
checkpoint_config_name = "nyctaxi_tripdata_yellow_yellow_tripdata__checkpoint"

In [0]:
# チェックポイントを定義
checkpoint_config = {
    "name":checkpoint_config_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": expectation_suite_name,
    "run_name_template": "%Y%m%d-%H%M%S-yctaxi_tripdata_yellow_yellow_tripdata",
}
context.add_checkpoint(**checkpoint_config)

Out[18]: {
 "action_list": [
 {
 "name": "store_validation_result",
 "action": {
 "class_name": "StoreValidationResultAction"
 }
 },
 {
 "name": "store_evaluation_params",
 "action": {
 "class_name": "StoreEvaluationParametersAction"
 }
 },
 {
 "name": "update_data_docs",
 "action": {
 "class_name": "UpdateDataDocsAction",
 "site_names": []
 }
 }
 ],
 "batch_request": {},
 "class_name": "Checkpoint",
 "config_version": 1.0,
 "evaluation_parameters": {},
 "expectation_suite_name": "nyctaxi_tripdata_yellow_yellow_tripdata",
 "module_name": "great_expectations.checkpoint",
 "name": "nyctaxi_tripdata_yellow_yellow_tripdata__checkpoint",
 "profilers": [],
 "run_name_template": "%Y%m%d-%H%M%S-yctaxi_tripdata_yellow_yellow_tripdata",
 "runtime_configuration": {},
 "validations": []
}

In [0]:
# checkpoints にファイルが作成されたことを確認
checkpoints_file_path = f'{root_directory_in_spark_api}/checkpoints/{checkpoint_config_name}.yml'
print(dbutils.fs.head(checkpoints_file_path))

name: nyctaxi_tripdata_yellow_yellow_tripdata__checkpoint
config_version: 1.0
template_name:
module_name: great_expectations.checkpoint
class_name: Checkpoint
run_name_template: '%Y%m%d-%H%M%S-yctaxi_tripdata_yellow_yellow_tripdata'
expectation_suite_name: nyctaxi_tripdata_yellow_yellow_tripdata
batch_request: {}
action_list:
 - name: store_validation_result
 action:
 class_name: StoreValidationResultAction
 - name: store_evaluation_params
 action:
 class_name: StoreEvaluationParametersAction
 - name: update_data_docs
 action:
 class_name: UpdateDataDocsAction
 site_names: []
evaluation_parameters: {}
runtime_configuration: {}
validations: []
profilers: []
ge_cloud_id:
expectation_suite_ge_cloud_id:

In [0]:
checkpoint_result = context.run_checkpoint(
    checkpoint_name=checkpoint_config_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name,
        }
    ],
)

Calculating Metrics: 0%| | 0/19 [00:00<?, ?it/s]

In [0]:
# uncommitted/validations にディレクトリが作成されたことを確認
checkpoints_file_path = f'{root_directory_in_spark_api}/uncommitted/validations/{expectation_suite_name}'
display(dbutils.fs.ls(checkpoints_file_path))

# uncommitted/data_docs/local_site にファイルとディレクトリが作成されたことを確認
checkpoints_file_path = f'{root_directory_in_spark_api}/uncommitted/data_docs/local_site'
display(dbutils.fs.ls(checkpoints_file_path))

path,name,size
file:/tmp/great_expectations/uncommitted/validations/nyctaxi_tripdata_yellow_yellow_tripdata/20220815-071849-yctaxi_tripdata_yellow_yellow_tripdata/,20220815-071849-yctaxi_tripdata_yellow_yellow_tripdata/,4096


path,name,size
file:/tmp/great_expectations/uncommitted/data_docs/local_site/index.html,index.html,34479
file:/tmp/great_expectations/uncommitted/data_docs/local_site/expectations/,expectations/,4096
file:/tmp/great_expectations/uncommitted/data_docs/local_site/validations/,validations/,4096
file:/tmp/great_expectations/uncommitted/data_docs/local_site/static/,static/,4096


### 検証結果を確認

In [0]:
# 品質チェック結果を表示
checkpoint_result["success"]

Out[22]: True

In [0]:
# 品質チェック結果の HTML ファイルをを表示
first_validation_result_identifier = (
    checkpoint_result.list_validation_result_identifiers()[0]
)
first_run_result = checkpoint_result.run_results[first_validation_result_identifier]

docs_path = first_run_result['actions_results']['update_data_docs']['local_site']

html = dbutils.fs.head(docs_path,)

displayHTML(html)

,
Evaluated Expectations,3
Successful Expectations,3
Unsuccessful Expectations,0
Success Percent,100%
,
Great Expectations Version,0.15.18
Run Name,20220815-071849-yctaxi_tripdata_yellow_yellow_tripdata
Run Time,2022-08-15T07:18:49Z
,
ge_load_time,20220815T071849.604061Z


## 品質エラーがある場合の動作検証

In [0]:
# エラーとなる expectation を追加
validator.expect_column_values_to_not_be_null(
    column="congestion_surcharge",
)

validator.save_expectation_suite(discard_failed_expectations=False)

In [0]:
checkpoint_result = context.run_checkpoint(
    checkpoint_name=checkpoint_config_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name,
        }
    ],
)

Calculating Metrics: 0%| | 0/20 [00:00<?, ?it/s]

In [0]:
# 品質チェック結果のを表示
checkpoint_result["success"]

Out[26]: False

In [0]:
# 品質チェック結果の HTML ファイルをを表示
first_validation_result_identifier = (
    checkpoint_result.list_validation_result_identifiers()[0]
)
first_run_result = checkpoint_result.run_results[first_validation_result_identifier]

docs_path = first_run_result['actions_results']['update_data_docs']['local_site']

html = dbutils.fs.head(docs_path,)

displayHTML(html)

,
Evaluated Expectations,4
Successful Expectations,3
Unsuccessful Expectations,1
Success Percent,75%
,
Great Expectations Version,0.15.18
Run Name,20220815-071957-yctaxi_tripdata_yellow_yellow_tripdata
Run Time,2022-08-15T07:19:57Z
,
ge_load_time,20220815T071957.465970Z


## データプロファイリング

次の記事を参考にしている。

- [Great Expectation on Databricks. Run great_expectations on the hosted… | by Probhakar | Medium](https://probhakar-95.medium.com/great-expectation-on-databricks-8777042e00de)

In [0]:
from great_expectations.profile.basic_dataset_profiler import BasicDatasetProfiler
from great_expectations.dataset.sparkdf_dataset import  SparkDFDataset
from great_expectations.render.renderer import *
from great_expectations.render.view import DefaultJinjaPageView

In [0]:
basic_dataset_profiler = BasicDatasetProfiler()

In [0]:
# creating GE wrapper around spark dataframe
from great_expectations.dataset.pandas_dataset import PandasDataset
gdf = PandasDataset(
    tgt_df
    .limit(1000)
    .toPandas()
) 

下記のコードにより、spark データフレームでも実行できるか、パフォーマンスに課題あり

```python
from great_expectations.dataset.sparkdf_dataset import SparkDFDataset
gdf = SparkDFDataset(
    tgt_df
    .limit(1000)
) 

print(gdf.spark_df.count())
gdf.spark_df.display()
```

In [0]:
# データを確認
print(gdf.count())
gdf.head()

VendorID 1000
tpep_pickup_datetime 1000
tpep_dropoff_datetime 1000
passenger_count 1000
trip_distance 1000
RatecodeID 1000
store_and_fwd_flag 1000
PULocationID 1000
DOLocationID 1000
payment_type 1000
fare_amount 1000
extra 1000
mta_tax 1000
tip_amount 1000
tolls_amount 1000
improvement_surcharge 1000
total_amount 1000
congestion_surcharge 0
dtype: int64
Out[31]:

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [0]:

from great_expectations.profile.basic_dataset_profiler import BasicDatasetProfiler

# データをプロファイリング
expectation_suite, validation_result = gdf.profile(BasicDatasetProfiler)

In [0]:
from great_expectations.render.renderer import (
    ProfilingResultsPageRenderer,
    ExpectationSuitePageRenderer,
)
from great_expectations.render.view import DefaultJinjaPageView

profiling_result_document_content = ProfilingResultsPageRenderer().render(validation_result)
expectation_based_on_profiling_document_content = ExpectationSuitePageRenderer().render(expectation_suite)

In [0]:
profiling_result_document_content

Out[34]: <great_expectations.render.types.RenderedDocumentContent at 0x7f3888702220>

In [0]:
# HTML を生成
profiling_result_HTML = DefaultJinjaPageView().render(profiling_result_document_content) # type string or str
expectation_based_on_profiling_HTML = DefaultJinjaPageView().render(expectation_based_on_profiling_document_content)

In [0]:
displayHTML(profiling_result_HTML)

,
Number of variables,18
Number of observations expect_table_row_count_to_be_between,1000
Missing cells,5.56%
,
int,6
float,9
string,1
datetime,2
bool,0
unknown,0


## リソースのクリーンアップ

In [0]:
dbutils.fs.rm(root_directory_in_spark_api, True)

Out[37]: True